### Import Requirements

In [87]:
import pandas as pd
import numpy as np
import gzip
import json
import re
import string
import nltk
import random

from sklearn.feature_extraction.text import TfidfVectorizer

!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

### Data Exploration

In [6]:
# Load Office Product Amazon Reviews as pandasDF
def parse(path):
    g = gzip.open(path, "rb")
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient="index")

data_path = "../Office_Products_5.json.gz"
df = getDF(data_path)

In [7]:
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4.0,True,"11 7, 2017",A2NIJTYWADLK57,0140503528,{'Format:': ' Board book'},cotton clay,kids like story BUT while i really wanted a bo...,"good story, small size book though",1510012800,NaN,NaN
1,4.0,True,"03 7, 2017",A2827D8EEURMP4,0140503528,{'Format:': ' Hardcover'},emankcin,Bought this used and it came in great conditio...,Good,1488844800,NaN,NaN
2,5.0,True,"06 25, 2016",APB6087F4J09J,0140503528,{'Format:': ' Board book'},Starbucks Fan,Every story and book about Corduroy is Fantast...,Best Books for All Children,1466812800,NaN,NaN
3,5.0,True,"02 21, 2016",A2DHERRZIPFU7X,0140503528,{'Format:': ' Paperback'},Caitlyn Jacobson,I purchased this book for my first grade class...,Great for Math!,1456012800,NaN,NaN
4,5.0,False,"08 2, 2015",A2XCLJRGFANRC,0140503528,{'Format:': ' Hardcover'},E. Ervin,Having spent numerous years in an elementary s...,Love Corduroy,1438473600,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
800352,5.0,True,"09 9, 2017",ACHG5QICJCS1F,B01HHLLLIO,{'Size:': ' 6 Rolls'},Anky,Delivered on time and is as expected.,5 stars,1504915200,NaN,NaN
800353,5.0,True,"06 2, 2017",A1YB5J8SF6QX3L,B01HHLLLIO,{'Size:': ' 6 Rolls'},DM,worked great.,good stuff,1496361600,NaN,NaN
800354,5.0,True,"04 29, 2017",A3N4O7L4HBYJX4,B01HHLLLIO,{'Size:': ' 6 Rolls'},Verdant Treasures,I used to score free UPS 4x6 labels. The blac...,Couldn't be happier with these labels,1493424000,NaN,NaN
800355,5.0,True,"02 21, 2017",A2DR3CXSWWLYXY,B01HHLLLIO,{'Size:': ' 6 Rolls'},C. F.,great,Five Stars,1487635200,NaN,NaN


This Dataset is an updated version of the Amazon review dataset released in 2014. As in the previous version, this dataset includes reviews (ratings, text, helpfulness votes), product metadata (descriptions, category information, price, brand, and image features), and links (also viewed/also bought graphs). In addition, this version provides the following features:

More reviews:
The total number of reviews is 233.1 million (142.8 million in 2014).
Newer reviews:
Current data includes reviews in the range May 1996 - Oct 2018.
Metadata:
We have added transaction metadata for each review shown on the review page. Such information includes:
Product information, e.g. color (white or black), size (large or small), package type (hardcover or electronics), etc.
Product images that are taken after the user received the product.
Added more detailed metadata of the product landing page. Such detailed information includes:
Bullet-point descriptions under product title.
Technical details table (attribute-value pairs).
Similar products table.
More categories:
Includes 5 new product categories.

Source: https://nijianmo.github.io/amazon/index.html#code

### Pre-processing

In [1]:
# preprocess reviewText here as well

In [3]:
# Plot the vector distribution (bar graph)

In [ ]:
# more preprocessing
## distribution graph
## limitization, stemming (week2)
## LaMDA ()?

In [19]:
# a. Randomly select 500-1000 reviews from your dataset and perform steps b through d.
n_samples = random.randint(500, 1000)
review_samples = df.sample(n=n_samples)
review_samples

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
778312,5.0,True,"07 9, 2018",A1OSFXOUGBKQQ5,B00BWU3HNY,{'Size:': ' 100-Pack'},Paul,They work like they are supposed to... I use t...,They work like they are supposed to,1531094400,NaN,NaN
528188,5.0,True,"03 17, 2016",A2W2JHM94N5GHC,B00DGDQUYI,NaN,C. Kaplan,Great product!,Five Stars,1458172800,NaN,NaN
88978,5.0,True,"05 1, 2015",A3MZ53HOJCQ2N,B00006JNJD,"{'Size:': ' Dozen Box', 'Color:': ' Blue', 'St...",MJS,"Hard to get excited over writing implements, b...",Really great pens.,1430438400,NaN,NaN
602569,5.0,True,"03 6, 2015",A3D58M29K2HWMV,B00MLUT4XQ,{'Color:': ' Blue-Lined'},P. Thompson,This just came today and really like it. Perfe...,Nice Little Journal,1425600000,6,NaN
290349,5.0,True,"12 4, 2016",A295498CGDJACR,B001602ZT2,{'Size:': ' 1 Pack'},Robert,Received as described and works as intended.,Five Stars,1480809600,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
249822,5.0,True,"08 30, 2016",A15T3Y3BL2AJ9V,B0017DCQCE,{'Size:': ' 1-Pack'},Roger Dodger,We go through a lot of pads like that. Some ar...,Very useful,1472515200,NaN,NaN
84642,4.0,True,"06 25, 2013",A2DJ97LLKUJ59R,B00006JNTT,NaN,Deb Shops,This is my second Swingline stapler. It beats ...,Best rechargeable stapler for the money,1372118400,NaN,NaN
479017,4.0,False,"10 11, 2014",AJN58R8C4UQVK,B008YD1V4Y,{'Style:': ' Printer'},Adam Hill,"everything: quality of prints, scanning, copyi...",happy with this printer,1412985600,NaN,NaN
198132,5.0,True,"02 15, 2011",A37IK59G4B1GAQ,B000SAD3R4,NaN,AlZer,everything was excellent,Five Stars,1297728000,NaN,NaN


In [23]:
#b. Label your data based on the value of “rating of the product” i.e. as follows:
    #i. Ratings 4,5: Positive
    #ii. Rating 3: Neutral
    #iii. Ratings 1,2: Negative

# Create new column
review_samples.insert(1, "sentiment", np.nan)
review_samples

,overall,sentiment,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
778312,5.0,NaN,True,"07 9, 2018",A1OSFXOUGBKQQ5,B00BWU3HNY,{'Size:': ' 100-Pack'},Paul,They work like they are supposed to... I use t...,They work like they are supposed to,1531094400,NaN,NaN
528188,5.0,NaN,True,"03 17, 2016",A2W2JHM94N5GHC,B00DGDQUYI,NaN,C. Kaplan,Great product!,Five Stars,1458172800,NaN,NaN
88978,5.0,NaN,True,"05 1, 2015",A3MZ53HOJCQ2N,B00006JNJD,"{'Size:': ' Dozen Box', 'Color:': ' Blue', 'St...",MJS,"Hard to get excited over writing implements, b...",Really great pens.,1430438400,NaN,NaN
602569,5.0,NaN,True,"03 6, 2015",A3D58M29K2HWMV,B00MLUT4XQ,{'Color:': ' Blue-Lined'},P. Thompson,This just came today and really like it. Perfe...,Nice Little Journal,1425600000,6,NaN
290349,5.0,NaN,True,"12 4, 2016",A295498CGDJACR,B001602ZT2,{'Size:': ' 1 Pack'},Robert,Received as described and works as intended.,Five Stars,1480809600,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249822,5.0,NaN,True,"08 30, 2016",A15T3Y3BL2AJ9V,B0017DCQCE,{'Size:': ' 1-Pack'},Roger Dodger,We go through a lot of pads like that. Some ar...,Very useful,1472515200,NaN,NaN
84642,4.0,NaN,True,"06 25, 2013",A2DJ97LLKUJ59R,B00006JNTT,NaN,Deb Shops,This is my second Swingline stapler. It beats ...,Best rechargeable stapler for the money,1372118400,NaN,NaN
479017,4.0,NaN,False,"10 11, 2014",AJN58R8C4UQVK,B008YD1V4Y,{'Style:': ' Printer'},Adam Hill,"everything: quality of prints, scanning, copyi...",happy with this printer,1412985600,NaN,NaN
198132,5.0,NaN,True,"02 15, 2011",A37IK59G4B1GAQ,B000SAD3R4,NaN,AlZer,everything was excellent,Five Stars,1297728000,NaN,NaN


In [33]:
# Label the sentimental values
for index, row in review_samples.iterrows():
    #i. Ratings 4,5: Positive
    if (row.overall >= 4.):
        print(index, "positive")
        review_samples.loc[index, ["sentiment"]] = "positive"
    #ii. Rating 3: Neutral
    elif (row.overall < 4. or row.overall > 2.):
        print(index, "neutral")
        review_samples.loc[index, ["sentiment"]] = "neutral"
    #iii. Ratings 1,2: Negative
    elif (row.overall <= 2.):
        print(index, "negative")
        review_samples.loc[index, ["sentiment"]] = "negative"

778312 positive
528188 positive
88978 positive
602569 positive
290349 positive
482083 positive
174249 positive
784744 positive
695568 positive
108220 positive
610966 positive
305476 positive
224798 positive
431436 positive
520845 positive
215445 positive
206744 positive
132820 positive
380653 positive
376160 positive
156875 positive
614754 positive
192142 positive
793608 positive
461050 positive
217810 neutral
5736 neutral
619101 neutral
741748 positive
162213 positive
533849 positive
73242 neutral
342975 positive
175151 neutral
470950 positive
286583 positive
233079 positive
491036 positive
778107 positive
61926 positive
448286 positive
96475 positive
368524 positive
791037 neutral
84268 positive
345900 positive
568391 positive
431013 positive
685179 positive
371439 positive
12177 positive
580761 neutral
538465 positive
251133 positive
167282 positive
33095 positive
628887 positive
789242 positive
388453 positive
650129 positive
34263 neutral
392250 positive
469641 positive
568283 pos

In [65]:
review_samples["sentiment"].isnull().any()

False

In [68]:
review_samples["reviewText"].isnull().any()

False

In [76]:
#c. Chose the appropriate columns for your sentiment analyzer. (Give this some thought)

### Text Representation

In [83]:
# Define methods to remove stopwords and punctuations from reviewText
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words("english")
ps = nltk.PorterStemmer()

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split("\W+", text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aiju2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [84]:
# Apply TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(review_samples["reviewText"])
print(X_tfidf.shape)
print(tfidf_vect.get_feature_names())

(542, 2618)
['', '03mm', '05', '05mm', '062pg', '07', '1', '10', '100', '1000', '1012', '10hour', '11', '114', '1162a', '11x14', '12', '127', '13', '14', '15', '150', '16', '18', '1982', '1year', '2', '20', '200', '2000', '2014', '2015', '23', '24', '25', '26', '2b', '3', '30', '300', '301', '307', '30odd', '30sec', '314', '32016', '3349a', '34', '35mm', '38', '388', '3m', '3ring', '4', '40', '48', '4999', '4pli', '4x12', '4x6', '4yo', '5', '50', '500', '53', '57', '5th', '6', '60', '61', '61xl', '62', '64bit', '6520', '682017', '6pack', '7', '701', '71', '74x', '75', '7inch', '7mm', '8', '80', '835', '845', '85x11', '8x10', '9', '90', '92', '93', '96', '9913', '9mm', 'abil', 'abl', 'absolut', 'absorbt', 'abysm', 'accept', 'acceptablebett', 'access', 'accid', 'accident', 'accommod', 'accordingli', 'acct', 'accur', 'ach', 'achiev', 'across', 'action', 'actual', 'ad', 'add', 'addit', 'address', 'adequ', 'adher', 'adhes', 'adivertis', 'adjac', 'adjust', 'adquat', 'adult', 'advanc', 'adver

### Modeling (Lexicon-based approach)

In [122]:
# Valence Aware Dictionary and Sentiment Reasoner (VADR)
VADR_analyzer = SentimentIntensityAnalyzer()
# Pass the analyzer for head 5 rows
for index, row in review_samples.head().iterrows():
    vs = VADR_analyzer.polarity_scores(row["reviewText"])
    print(
        f"Index: {index}\n"+
        f"Sentimental Analysis Result: {vs}\n"+
        f"Overall rating category: {row.sentiment}\n"+
        f"Full Text:\n{row.reviewText}\n"+
        "-"*50)

Index: 778312
Sentimental Analysis Result: {'neg': 0.0, 'neu': 0.857, 'pos': 0.143, 'compound': 0.3612}
Overall rating category: positive
Full Text:
They work like they are supposed to... I use them in my work as a teacher...
--------------------------------------------------
Index: 528188
Sentimental Analysis Result: {'neg': 0.0, 'neu': 0.185, 'pos': 0.815, 'compound': 0.6588}
Overall rating category: positive
Full Text:
Great product!
--------------------------------------------------
Index: 88978
Sentimental Analysis Result: {'neg': 0.028, 'neu': 0.781, 'pos': 0.191, 'compound': 0.7814}
Overall rating category: positive
Full Text:
Hard to get excited over writing implements, but these are my personal favorite pens.  Can't speak for the pricing vs. local stores, but they showed up in one day and the convenience is worth a small upcharge.
--------------------------------------------------
Index: 602569
Sentimental Analysis Result: {'neg': 0.039, 'neu': 0.828, 'pos': 0.133, 'compound':

### Validation